In [3]:
import pprint
import os
import sys
from params import get_cfg
from rnn_base import RNN
from ops import Operations
from session import run_session, restore_selection_matrixes
from data_gen import samples_generator, split_train_test
import tensorflow as tf

#get the global configuration
cfg = get_cfg() 
#instantiate containter with the operations avail for the selection
ops = Operations(cfg)
#reset graph so that tf dont complain about not found vars
tf.reset_default_graph()
#generate data 
x,y = samples_generator(cfg['train_fn'], (cfg['num_samples'], cfg['num_features']) , cfg['samples_value_rng'], cfg['seed'])
x_train, x_test, y_train, y_test = split_train_test (x, y , cfg['test_ratio'])
# instanitae the model graph
model = RNN(cfg, ops)
#restore with given path
path = './summaries/np_add-5ops/test1/model/'
_softmaxes_train, _softmax_train, _softmaxes_test, _softmax_test = restore_selection_matrixes(model, cfg, x_train, x_test, y_train, y_test, path)


timestep 0
timestep 1
timestep 2
timestep 3
timestep 4
timestep 0
timestep 1
timestep 2
timestep 3
timestep 4
[<tf.Variable 'W:0' shape=(53, 50) dtype=float64_ref>, <tf.Variable 'b:0' shape=(50,) dtype=float64_ref>, <tf.Variable 'W2:0' shape=(50, 3) dtype=float64_ref>, <tf.Variable 'b2:0' shape=(3,) dtype=float64_ref>]
norming the grads
grads are
[<tf.Tensor 'clip_by_global_norm/clip_by_global_norm/_0:0' shape=(53, 50) dtype=float64>, <tf.Tensor 'clip_by_global_norm/clip_by_global_norm/_1:0' shape=(50,) dtype=float64>, <tf.Tensor 'clip_by_global_norm/clip_by_global_norm/_2:0' shape=(50, 3) dtype=float64>, <tf.Tensor 'clip_by_global_norm/clip_by_global_norm/_3:0' shape=(3,) dtype=float64>]
num batches train: 10
num batches test: 4
INFO:tensorflow:Restoring parameters from ./summaries/np_add-5ops/test1/model/-1950


NameError: name 'softmaxes_train' is not defined

In [ ]:
for softmaxe in _softmaxes_train:
    print("#####################################################################")
    print(softmaxe)

In [ ]:
for softma in _softmaxes_test:
    print("#####################################################################")
    print(softma)

In [ ]:
_softmax_train.shape

In [ ]:
W2.shape

In [ ]:
W.shape

In [ ]:
init_state.shape

In [ ]:
b.shape

In [ ]:
b2.shape